<!-- Autoheader begin -->
<hr/>
<div id="navtitle_2_3_py" style="text-align:center; font-size:16px">II.3 Parameter Optimization of Three-Wave Mixing in a Three-Level System</div>
<hr/>
<table style="width: 100%">
  <tr>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_2_2_lambda.ipynb">$\leftarrow$ previous notebook </a><br>
        <a href="py_exercise_2_2_lambda.ipynb" style="font-size:13px">II.2 Parameter Optimization for STIRAP</a>
    </th>
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_1_3_chirp.ipynb">$\uparrow$ previous part $\uparrow$</a><br>
        <a href="py_exercise_1_3_chirp.ipynb" style="font-size:13px">I.3 Interaction of a Two-Level-System with a Chirped Laser Pulse</a>
    </td>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_3_1_TLS.ipynb">next notebook $\rightarrow$</a><br>
        <a href="py_exercise_3_1_TLS.ipynb" style="font-size:13px">III.1 Population Inversion in a Two-Level-System using Krotov's Method</a>
    </th>
  </tr>
  <tr style="width: 100%">
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_3_3_chiral.ipynb" style="font-size:13px">III.3 Using Krotov's method to separate chiral molecules</a><br>
        <a href="py_exercise_3_3_chiral.ipynb">$\downarrow$ next part $\downarrow$</a>
    </td>
  </tr>
</table>

<div style="text-align: right;font-size: 16px"><a href="../Julia/jl_exercise_2_3_chiral.ipynb">👉 Julia version</a></div>

---
<!-- Autoheader end -->

# Parameter Optimization of Three-Wave Mixing in a Three-Level System

$
\newcommand{tr}[0]{\operatorname{tr}}
\newcommand{diag}[0]{\operatorname{diag}}
\newcommand{abs}[0]{\operatorname{abs}}
\newcommand{pop}[0]{\operatorname{pop}}
\newcommand{aux}[0]{\text{aux}}
\newcommand{opt}[0]{\text{opt}}
\newcommand{tgt}[0]{\text{tgt}}
\newcommand{init}[0]{\text{init}}
\newcommand{lab}[0]{\text{lab}}
\newcommand{rwa}[0]{\text{rwa}}
\newcommand{bra}[1]{\langle#1\vert}
\newcommand{ket}[1]{\vert#1\rangle}
\newcommand{Bra}[1]{\left\langle#1\right\vert}
\newcommand{Ket}[1]{\left\vert#1\right\rangle}
\newcommand{Braket}[2]{\left\langle #1\vphantom{#2} \mid #2\vphantom{#1}\right\rangle}
\newcommand{op}[1]{\hat{#1}}
\newcommand{Op}[1]{\hat{#1}}
\newcommand{dd}[0]{\,\text{d}}
\newcommand{Liouville}[0]{\mathcal{L}}
\newcommand{DynMap}[0]{\mathcal{E}}
\newcommand{identity}[0]{\mathbf{1}}
\newcommand{Norm}[1]{\lVert#1\rVert}
\newcommand{Abs}[1]{\left\vert#1\right\vert}
\newcommand{avg}[1]{\langle#1\rangle}
\newcommand{Avg}[1]{\left\langle#1\right\rangle}
\newcommand{AbsSq}[1]{\left\vert#1\right\vert^2}
\newcommand{Re}[0]{\operatorname{Re}}
\newcommand{Im}[0]{\operatorname{Im}}
$

This notebook performs optimization of three-wave mixing in a chiral three-level system.
In this notebook you will learn how to use gradient-free parameter optimization via the `NLopt` package for the purpose of driving two three-level systems, representing the two mirror images (the so-called enantiomers) of a chiral molecules, such that they end up in distinct final states. This allows for the discrimination of enantiomers which is a central task in applications involving chiral molecules.

## Setup

We start with importing the necessary Python packages, among which are
* `qutip`: a package for quantum modeling and dynamics
* `nlopt`: an optimization package

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
pi = np.pi
from qutip import *
import nlopt

# Some utilities for showing hints and solutions
from utils.exercise_2_chiral import *

## Model

<img src="../figures/3-level_mod.svg" alt="Drawing" style="width: 800px;"/>

The illustration above shows two a three-level model for a chiral molecules with the left and right side corresponding to the $+$(left), respectively $-$(right), enantiomer of the molecule.
The three levels are connected with each other by an electric dipole transition. The only difference between the two enantiomers lies in the sign of the $\mu_{c}$ component of the dipole transition moment. Our goal is to obtain 'enantioselectivity' which means that a sequence of microwave pulses applied to the two enantiomers starting in the same initial state will lead to two perfectly distinct final states.

## Hamiltonian
We begin by defining the function `total_enantiomer_ham`. This function returns the Hamiltonian (in the rotating frame) for the two enantiomers, i.e.,

$$
\hat{H}_{I}^{(\pm)}(t) = \sum_{i=1}^{3} \hat{H}_{i}^{(\pm)}(t)
$$

with

$$
\hat{H}_{1}^{(\pm)}(t)
=
- E_{1}(t) \mu_{b}^{(\pm)}
\begin{pmatrix}
  0 & e^{i \phi_{1}} & 0 \\
  e^{- i \phi_{1}} & 0 & 0 \\
  0 & 0 & 0 \\
\end{pmatrix},
\\
\hat{H}_{2}^{(\pm)}(t)
=
- E_{2}(t) \mu_{a}^{(\pm)}
\begin{pmatrix}
  0 & 0 & 0 \\
  0 & 0 & e^{i \phi_{2}} \\
  0 & e^{- i \phi_{2}} & 0 \\
\end{pmatrix},
\\
\hat{H}_{3}^{(\pm)}(t)
=
- E_{3}(t) \mu_{c}^{(\pm)}
\begin{pmatrix}
  0 & 0 & e^{i \phi_{3}} \\
  0 & 0 & 0 \\
  e^{- i \phi_{3}} & 0 & 0 \\
\end{pmatrix}
$$

and where

$$
E_{i}(t)
=
\frac{E_{i,0}}{2}
\left[\tanh(a (t - t_{i,1})) - \tanh(a (t - t_{i,2}))\right]
$$

for $i \in \left\{1,2,3\right\}$ are the envelopes of the pulses with frequencies $\omega, \delta \omega$ and $\omega + \delta \omega$.

The function takes three different sets of input parameters, which altogether specify the Hamiltonian and the control fields in their entirety. The three sets are, respectively:

* `pulse_durations`: A list of the three durations $\Delta t_{i} = t_{i,2} - t_{i,1}$ of each of the three fields $E_{i0}(t)$. Field $i$ is assumed to start when field $i-1$ ends. The first field starts at $t_{1,1}=0$.
* `phis`: A list of the three real phases $\phi_{i}$ for each field.
* `Ei0s`: A list of the three real amplitudes $E_{i0}$ for each field.
* `sign`: The string `+` or `-` specifies which Hamiltonian, i.e., $H_{I}^{(+)}(t)$ or $H_{I}^{(-)}(t)$, is retuned.
* `a`: This is a parameter which controls how smooth each field is turned on and off. The larger `a` becomes, the more the field shapes $E_{i0}(t)$ resemble a rectangle.

In [ ]:
# lists in python begin at index 0, so this shift is to offset the level indices for the list addresses
shift = -1
def total_enantiomer_ham(pulse_durations, phis, Ei0s, sign, a):
    H_i_list = []

    # i is the index of the currently treated level
    for i in [1,2,3]:
        # select the right phi_i, E_i0 and mu_i for Hamiltonian H_i
        phi_i = phis[shift+i]
        phase = np.exp(1j*phi_i)
        phase_dag = np.conj(phase)

        E_i0 = Ei0s[i-1]
        mu_i = -1. if (sign == '-' and i == 3) else 1.

        # define Hamiltonian H_i by connecting levels and pulses according to
        # pulse 1: 1 <-> 2
        # pulse 2: 2 <-> 3
        # pulse 3: 1 <-> 3
        if i == 1:
            H_i = mu_i*Qobj([
                [        0,    phase,        0],
                [phase_dag,        0,        0],
                [        0,        0,        0]])
        if i == 2:
            H_i = mu_i*Qobj([
                [        0,        0,        0],
                [        0,        0,    phase],
                [        0,phase_dag,        0]])
        if i == 3:
            H_i = mu_i*Qobj([
                [        0,        0,    phase],
                [        0,        0,        0],
                [phase_dag,        0,        0]])

        # define the start and end time of the field that couples to Hamiltonian H_i
        t_i1 = np.sum(pulse_durations[:shift+i])
        t_i2 = t_i1 + pulse_durations[shift+i]

        # specify the form of the control field
        def guess_control(E_i0, a, t_i1, t_i2):
            return lambda t,args : 0.5*E_i0*(np.tanh(a*(t-t_i1))-np.tanh(a*(t-t_i2)))

        H_i_list.append([H_i, guess_control(E_i0, a, t_i1, t_i2)])

    return H_i_list

We define the initial state as

$$
\Psi_{\pm}(0)
=
\begin{pmatrix}
  1 \\ 0 \\ 0
\end{pmatrix}.
$$

In [ ]:
# the initial state consists of three levels with population initially in the ground state
psi_initial = Qobj([
    [1],
    [0],
    [0]])

Our time grid is obtained by dividing $$t \in \left[0,1\right]$$ into 100 equal intervals (remember that 100 intervals implies 101 points of the time grid).

In [ ]:
tlist = np.linspace(0., 1., 101)

## Problem 0 - Pulse parameterisation
We begin by familiarizing ourselves with the pulse parameterization. In this notebook the pulses are formed by the difference between two hyperbolic tangent functions.
`E0` controls the pulse amplitude, `a` controls how rectangular the pulse appears and `t_start` and `t_stop` determine when the pulse starts and ends.
Try changing the arguments of `plot_parameterised_pulse`, such that the two curves match (an exact fit is difficult, it is sufficient to aim for a value of the calculated mismatch which is below one).

In [ ]:
def plot_parameterised_pulse(E0,a,t_start,t_stop):
    tanh_pulse = 0.5 * E0 * (np.tanh(a*(tlist-t_start))-np.tanh(a*(tlist-t_stop)))
    def target_pulse():
        return 20*np.exp(-20*(tlist-0.5)**2)
    mismatch = np.linalg.norm(tanh_pulse - target_pulse(), ord=1) / len(tlist)
    plt.plot(tlist,tanh_pulse,label='your pulse', c='g' if mismatch < 1 else 'b')
    plt.plot(tlist,target_pulse(),label='target pulse',c='orange')
    plt.text(0.,20,'mismatch: {:8.4f}'.format(mismatch))
    plt.legend()

plot_parameterised_pulse(E0=3.14,a=500,t_start=0.15,t_stop=0.85)

In [ ]:
#problem_0.hint

In [ ]:
#problem_0.solution

## Initialise optimization

Now we can turn towards optimization. To this end, we use the optimization methods provided by the `nlopt` package. For simplicity, we use the well-known Nelder-Mead method. Note, however, that `nlopt` allows for a wide range of different methods.

The `nlopt` for optimization requires a `loss` function, i.e. the optimization functional, which takes a set of real-valued parameters (called `x` here). One may also provide a function for the gradient but this is only required if a gradient-based optimization method is employed, which we do not study in this notebook on gradient-free optimization. Hence, we set the gradient to `None`.

The `loss` function takes the list `x`, which contains our optimization parameters, i.e., all pulse durations, all phases $\phi_{i}$ and all amplitudes $E_{i,0}$, on input and returns the error of the enantioselectivity protocol introduced above. Specifically, `loss` returns zero if and only if the dynamics obtained from the parameters in the set `x` transfers the initial state $\Psi(0)$ perfectly into the target state, i.e.,

$$
\Psi_{+}(0)
\longrightarrow
\Psi_{+}\left(T\right) =
\begin{pmatrix}
  1 \\ 0 \\ 0
\end{pmatrix}
$$

for enantiomer `+` and

$$
\Psi_{-}(0)
\longrightarrow
\Psi_{-}\left(T\right) =
\begin{pmatrix}
  0 \\ 0 \\ 1
\end{pmatrix}
$$

for enantiomer `-`.

In [ ]:
def population(state,level):
    return abs(state[shift+level][0][0])**2

def loss(x, grad=None):
    global iterations
    global obtained_fidelities
    iterations += 1
    # map the optimization parameters from the set x to the three individual sets for times, phases and amplitudes
    pulse_durations = [x[0],x[1],x[2]]
    phis = [x[3],x[4],x[5]]
    Ei0s = [x[6],x[7],x[8]]

    # define the two Hamiltonians for the two enantiomers
    Htot_plus = total_enantiomer_ham(pulse_durations, phis, Ei0s, '+', a=1000)
    Htot_minus = total_enantiomer_ham(pulse_durations, phis, Ei0s, '-', a=1000)

    # simulate the dynamics
    results_plus = sesolve(Htot_plus, psi_initial, tlist)
    results_minus = sesolve(Htot_minus, psi_initial, tlist)

    # define the fidelity of the outcome
    def fid(psi_plus, psi_minus):
        return (population(psi_plus,level=1) + population(psi_minus,level=3))/2

    # calculate the fidelity for the dynamics under the parameters from set x
    fidelity = fid(results_plus.states[-1],results_minus.states[-1])
    obtained_fidelities.append(fidelity)

    print(f"Iteration: {iterations:}, current fidelity {fidelity:8.4f}", end="\r")
    # return the error (this is our optimization functional)
    return 1. - fidelity

## Problem 1 - Run optimization

Next, we can run the actual optimization. It requires us to define lower and upper bounds for all parameters that should be optimized. In our case, we choose

$$
0 \leq t_{i,2} \leq T=1,
\qquad
0 \leq \phi_{i} \leq 2 \pi,
\qquad
0 \leq E_{i,0} \leq 10.
$$

Note that we need to provide guess values for all nine parameters. The order that the parameters should appear in is:
* `pulse_durations`
* `phis`
* `Ei0s`

The choice of the guess values will often have an appreciable impact on the general success of the optimized solution and can even affect its form since many optimization problems allow for many different solutions.
Your task is now to fill in the upper optimization bounds and to try different
guesses to evaluate their impact on the optimization.

In [ ]:
# define lower and upper bounds for three pulse durations, three phis and three E0s that are to be optimized
bounds_lower = [0,0,0, 0,0,0, 0,0,0]
bounds_upper = [insert_upper_bounds_here]

## determine guess parameters for all parameters that are optimized
# set guess parameters for all parameters that are optimized
guess = [insert_guess_parameters_here]

# specify the optimization method and the number of parameters that should be optimized
# (given here by len(bounds_lower))
opt = nlopt.opt(nlopt.LN_NELDERMEAD, len(bounds_lower))

# set the lower and upper bound for the optimization
opt.set_lower_bounds(bounds_lower)
opt.set_upper_bounds(bounds_upper)
opt.set_maxeval(500)

# set the objective that should be optimized (note that this implies minimisation)
opt.set_min_objective(loss)

# fidelity above which the optimization should be stopped
fid_min = 0.99

# define below which value, i.e., error, to stop the optimization
opt.set_stopval(1.-fid_min)

# perform the optimization
obtained_fidelities = []
iterations = 0
x = opt.optimize(guess)

print('\n\nHighest fidelity reached: {:6.2f}%'.format(100.0*(1-opt.last_optimum_value())))
if opt.last_optimum_value() > 1-fid_min:
    print('\tbad guess, please try again!')
else:
    print('\tcongratulations, you have obtained enantio-selectivity!')

In [ ]:
#problem_1.hint

In [ ]:
#problem_1.solution

In [ ]:
# plot the results
plt.plot(range(len(obtained_fidelities)),obtained_fidelities,marker='x')
plt.xlabel('optimization iterations');
plt.ylabel('fidelity');

## Analyze optimization results
After optimiziation we can verify the optimization result by plotting the pulses
as well as the resulting dynamics.
To this end, we define the Hamiltonians of the two enantiomers given the optimized parameters.

In [ ]:
Htot_plus  = total_enantiomer_ham([x[0], x[1], x[2]], [x[3], x[4], x[5]], [x[6], x[7], x[8]], '+', a=1000)
Htot_minus = total_enantiomer_ham([x[0], x[1], x[2]], [x[3], x[4], x[5]], [x[6], x[7], x[8]], '-', a=1000)

In order to visualize the optimized pulses, we plot them in the following.

In [ ]:
tlist = np.linspace(0., 1., 500)
def plot_pulse(pulses, tlist):
    fig, ax = plt.subplots()
    for i, pulse in enumerate(pulses):
        if callable(pulse):
            pulse = np.array([pulse(t, args=None) for t in tlist])
        ax.plot(tlist, pulse, label=r"$E_{}$".format(i+1))
    plt.xlabel("time")
    plt.ylabel("amplitude")
    plt.legend()
    plt.show(fig)

plot_pulse([Htot_plus[0][1], Htot_plus[1][1], Htot_plus[2][1]], tlist)

Finally, we solve the dynamics of the two enantiomers (now using the optimized parameters) and plot their population dynamics.

In [ ]:
def propagate_system(Htot, title):
    # simulate the dynamics determined by Htot
    results = sesolve(Htot, psi_initial, tlist)

    # collect the arrays for the population in the three levels
    c_1 = [(abs(state[0])**2).item() for state in results.states]
    c_2 = [(abs(state[1])**2).item() for state in results.states]
    c_3 = [(abs(state[2])**2).item() for state in results.states]
    # plot the population dynamics
    plt.plot(tlist, c_1, label=r"$\left\|c_1\right|^2$")
    plt.plot(tlist, c_2, label=r"$\left\|c_2\right|^2$")
    plt.plot(tlist, c_3, label=r"$\left\|c_3\right|^2$")
    plt.xlabel("time")
    plt.ylabel("population")
    plt.title(title)
    plt.legend()
    plt.show()

# plot the population dynamics for both enantiomers
propagate_system(Htot_plus, "enantiomer +")
propagate_system(Htot_minus, "enantiomer -")

In the plots above you see how the population evolves under the influence of our parameterised pulses. If the optimization was successful, the populations of the two enantiomers at the final time $T$ should be entirely in the $\Psi_{+}(T)$ (first level) and $\Psi_{-}(T)$ (third level) target states, respectively.

## Next steps

This notebook is our final example for parameter optimization. Now you are ready to learn about gradient-based optimization. [Exercise III.2](py_exercise_3_2_lambda.ipynb) gives an introduction to this concept for the example of the STIRAP protocol. Alternatively, if you would like to stick with chiral molecules and three-wave-mixing, have a look at [Exercise III.3](py_exercise_3_3_chiral.ipynb), in which gradient-based opimization is employed for the problem you studied in this notebook.

<!-- Autofooter begin -->

---

[⬆︎ jump to top](#navtitle_2_3_py)
<!-- Autofooter end -->